#__Credit Scoring__
Credit scoring is used by lenders to decide on whether to extend or deny credit based on person's credit score.
This project mostly aimed to data cleaning. Bank provided the dataset that contained information about bank's clients solvency. 


To improve the bank's scoring model we've been asked to analyse if there is a correlation between marital status and parenting of a borrower and due credit repayment. 

## __Data Review__

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
customer_solvency = pd.read_csv('/content/data.csv')

In [3]:
customer_solvency.info()
customer_solvency.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
# Check data for dublicates
customer_solvency.duplicated().sum()

54

In [5]:
# Check data for missing values
customer_solvency.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Let's check for any anomalies in our data

In [6]:
# Number of children
customer_solvency.sort_values('children')['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

The number of childre as "-1" and "20" seems strange, something wrong

In [7]:
# Check if the column "days_employed" has any negative numbers
customer_solvency[customer_solvency['days_employed'] < 0]['days_employed'].count()

15906

Wow, not good, we have a lot of negative values in the column where it's not supposed to be

In [8]:
# Check positive numbers in the column "days_employed"
customer_solvency[customer_solvency['days_employed'] > 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


It looks strange, positive numbers are too big. For example, if we divide first raw 340266 by 365 we get 340266/365 = **~932 years**. Well, either many of bank's clients are immortal or we've got an error in these values. I am inclined to the second option. But, to be safe, let's check clients' age as well.

In [9]:
customer_solvency.sort_values('dob_years')['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Nope, unfortunately (or fortunately) the clients are just normal humans with quite normal age. However, we also see that 101 client is __0 years__ old. Obviously, it's an error.

In [10]:
# Check an information about education
customer_solvency.sort_values('education')['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [11]:
# Checking marital status
customer_solvency['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [12]:
# Check gender data
customer_solvency['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [13]:
# Check an income type
customer_solvency['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

__Income. Dive Deeper__

In [14]:
# Check if we have clients with zero income
customer_solvency[customer_solvency['total_income'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [15]:
# Check missing values re income
customer_solvency['total_income'].isnull().sum()

2174

In [16]:
# Check loan purposes
customer_solvency['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

##__Data Preprocessing__

We've been asked if there is a correlation between marital status and parenting of a borrower and due credit repayment.
Therefore, we will focus only on the columns that are useful for our goal

In [17]:
# Create mapping for filing missing values in "days_employed"
median_values = pd.pivot_table(
    customer_solvency,
    index=['education', 'income_type'],
    values=['days_employed', 'total_income'],
    aggfunc=np.median
)

median_values

days_employed   total_income
education           income_type                                  
ВЫСШЕЕ              госслужащий       -1424.562274  174909.179988
                    компаньон         -1423.978136  200701.997023
                    пенсионер        367532.282115  165563.164062
                    сотрудник         -1602.753306  144683.271465
Высшее              безработный      395302.838654  202722.511368
                    госслужащий       -3750.610464  161671.347625
                    компаньон         -1453.451901  192451.667511
                    пенсионер        368379.850772  154962.528710
                    сотрудник         -1534.735406  178238.955336
НАЧАЛЬНОЕ           компаньон         -1265.943306  190952.908390
                    пенсионер        360264.985350   90479.336389
                    сотрудник         -1012.313022  212545.366151
НЕОКОНЧЕННОЕ ВЫСШЕЕ компаньон          -775.760267  145528.679914
                    пенсионер        358732.475441  412386.347106
                    сотрудник         -1546.148345  166015.790504
Начальное           госслужащий        -301.333707  190912.178349
                    компаньон         -1030.219648   88227.830722
                    пенсионер        364632.447491   98853.333449
                    сотрудник          -852.697747  129895.001992
Неоконченное высшее госслужащий       -1178.018163  107224.860090
                    компаньон          -711.837088  183379.320368
                    пенсионер        396568.887159   83625.583859
                    сотрудник         -1653.837028  159115.726175
СРЕДНЕЕ             в декрете         -3296.759962   53829.130729
                    госслужащий       -3356.097160  132756.053048
                    компаньон         -1945.914306  157308.365600
                    пенсионер        363541.465259  113656.027279
                    сотрудник         -1567.112656  135823.934197
Среднее             госслужащий       -2291.747052  148529.629601
                    компаньон         -1863.730566  149190.998579
                    пенсионер        363995.316389  128155.229215
                    сотрудник         -1730.667138  133399.107243
УЧЕНАЯ СТЕПЕНЬ      сотрудник          -409.200149  198570.757322
Ученая степень      пенсионер        337584.815560   98752.495442
высшее              госслужащий       -2577.299850  172511.107016
                    компаньон         -1461.167287  201911.716329
                    пенсионер        366157.236636  142825.046222
                    предприниматель    -520.848083  499163.144947
                    сотрудник         -1548.646517  166164.078024
                    студент            -578.751554   98201.625314
начальное           госслужащий       -3043.933615  105766.403301
                    компаньон         -1166.810199  170077.891115
                    пенсионер        358777.901745  104505.757848
                    сотрудник         -1261.719742  124298.759469
неоконченное высшее госслужащий       -2038.132156  165106.669771
                    компаньон         -1032.968329  179867.152890
                    пенсионер        371193.915404  122706.716059
                    сотрудник         -1158.032596  149551.191876
среднее             безработный      337524.466835   59956.991984
                    госслужащий       -2856.781570  136652.970357
                    компаньон         -1645.331963  159652.900458
                    пенсионер        365232.442222  114483.373934
                    сотрудник         -1607.383724  136818.115423
ученая степень      госслужащий       -5968.075884  111392.231107
                    пенсионер        376276.219531  255425.196556
                    сотрудник         -3851.735057  192180.127162

In [0]:
# Create a function to fill in the missing values in 'days_employed' column
def fill_days_employed(row):
    return median_values.loc[row['education']].loc[row['income_type']]['days_employed']

In [0]:
customer_solvency.loc[
    customer_solvency['days_employed'].isnull(), 'days_employed'
] = customer_solvency.apply(fill_days_employed, axis=1)

In [23]:
customer_solvency['days_employed'].isnull().sum()

0

In [0]:
# Create a function to fill in the missing values in 'total_income' column
def fill_total_income(row):
    return median_values.loc[row['education']].loc[row['income_type']]['total_income']

In [0]:
customer_solvency.loc[
    customer_solvency['total_income'].isnull(), 'total_income'
] = customer_solvency.apply(fill_total_income, axis=1)

In [26]:
customer_solvency['total_income'].isnull().sum()

0

In [34]:
customer_solvency[customer_solvency['children'] > 5]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,-1645.331963,53,среднее,1,женат / замужем,0,M,компаньон,0,159652.900458,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


In [36]:
# Remove values that represent more than 5 children (actually 20 children, which is definitely an error)
customer_solvency = customer_solvency[(customer_solvency['children'] >= 0) & (customer_solvency['children'] <= 5)]
customer_solvency.sort_values('children')['children'].value_counts().sort_index()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [38]:
customer_solvency.shape

(21402, 12)

In [40]:
# Replace float values by absolute values
customer_solvency['days_employed'] = customer_solvency['days_employed'].astype('int').apply(abs)
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [0]:
# Remove dublicates by unification of register
customer_solvency['education'] = customer_solvency['education'].str.lower()

In [42]:
customer_solvency.sort_values('education')['education'].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

##__Analysis of correlation between parenting and solvency__

Let's categorize our dataset into three categories based on number of children:

* 0 – бездетная (no children)
* 1 – однодетная (one-child family)
* 2 – малодетная (small family)
* 3+ – многодетная (large family)

In [0]:
def categorize_children(count):
    categories = {
        0: 'бездетная',
        1: 'однодетная',
        2: 'малодетная'
    }

    return categories.get(count, 'многодетная')

In [56]:
customer_solvency['children_category'] = customer_solvency['children'].apply(categorize_children)
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,однодетная
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,однодетная
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,бездетная
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,многодетная
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,бездетная


In [61]:
pd.pivot_table(customer_solvency, index='children_category', values='debt')

,debt
children_category,
бездетная,0.075129
малодетная,0.094404
многодетная,0.081579
однодетная,0.092154


__Conclusion:__ there is no strict correlation between the number of children and solvency, except for "no-child" families. They have a little bit less debt than others

##__Analysis of correlation between marital status and solvency__

In [0]:
# Categorize family status as "married/not-married"
def categorize_family_status(value):
    if value == 'женат / замужем':
        return 'в браке'
    
    return 'не в браке'

In [0]:
customer_solvency['family_status_category'] = customer_solvency['family_status'].apply(categorize_family_status)

In [60]:
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,однодетная,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,однодетная,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,бездетная,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,многодетная,в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,бездетная,не в браке


In [62]:
pd.pivot_table(customer_solvency, index='family_status_category', values='debt')

,debt
family_status_category,
в браке,0.075354
не в браке,0.088462


__Conclusion:__ again, the difference between single and married debtors are not significant, only 1%